In [1]:
import os
# Specify the new directory path
parent_directory = 'D:/Projects/CornYield/src'

# Change the current directory
os.chdir(parent_directory) 
# %pwd

In [2]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split

In [3]:
from dataset import select_data_and_yield_list
from dl.dl_dataset import CornDatasetTimeSeries
from dl.model import ConvLSTMRegression
from dl.train import train_with_cross_validation, train, validate, data_transform

In [4]:
yield_file = 'D:/Corn_Yield/BL2022_Yld.csv'
img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220610_DOY161_extracted_filled'
out_path = '../output/DOY161/'
# img_path = 'D:/Corn_Yield/UAV_Data_Extracted_filled/LIRF20220926_DOY269_extracted_filled'
# out_path = '../output/DOY269/'
if not os.path.exists(out_path):
    # If it doesn't exist, create the directory and any missing parent directories
    os.makedirs(out_path)
    print(f"Directory '{out_path}' created successfully.")
else:
    print(f"Directory '{out_path}' already exists.")


name_tag = img_path[-23:-17]
# print(name_tag)

# key_word_list = ['Ref_filled.tif', 'RGB_filled.tif']
key_word_list = ['Ref_filled.tif']
suffix_list_list = [[], ['LWIR_filled.tif']]

# suffix = ['base', 'lwir']
# suffix_list = ['LWIR_filled.tif']
# VI_list = ['ndvi', 'ndre', 'gndvi', 'evi']
VI_list = ['evi']
for keyword in key_word_list:

    selection = ['Pioneer'] # 
    # selection = 'Pioneer Deficit' 
    # selection = 'Pioneer Full'
    pioneer_img_list, pioneer_yield_list, irrigate_type_list = select_data_and_yield_list(
        img_path, yield_file, key_word = keyword, crop_type_select=selection)
    
    
    # for suffix_list in suffix_list_list:
    
    total_size = len(pioneer_yield_list)
    train_size = int(0.8 * total_size)  # 80% for training
    test_size = int(0.2 * total_size)   # 20% for validation
    
    # g = torch.Generator()
    # g.manual_seed(39)
    
    # train_val_dataset, test_dataset = random_split(all_dataset, [train_size, test_size])
    
    # # all_dataset = CornDataset(pioneer_img_list, pioneer_yield_list, transform = transform)
    # all_dataset = CornDatasetTimeSeries(pioneer_img_list, pioneer_yield_list, transform=data_transforms)
    
    # # Use train_test_split to split the indices into training and testing sets
    # train_indices, test_indices = train_test_split(range(len(all_dataset)), test_size=test_size, random_state=39)
    # train_val_dataset = torch.utils.data.Subset(all_dataset, train_indices)
    # test_dataset = torch.utils.data.Subset(all_dataset, test_indices)
    
   
    # Use train_test_split to split the indices into training and testing sets
    train_indices, test_indices = train_test_split(range(len(pioneer_img_list)), test_size=test_size, random_state=39)

    train_val_dataset = CornDatasetTimeSeries([pioneer_img_list[i] for i in train_indices], [pioneer_yield_list[i] for i in train_indices], transform=data_transform())
    test_dataset = CornDatasetTimeSeries([pioneer_img_list[i] for i in test_indices], [pioneer_yield_list[i] for i in test_indices])

    

Directory '../output/DOY161/' already exists.


In [5]:
print(len(train_val_dataset))
print(type(train_val_dataset))

3192
<class 'dl.dl_dataset.CornDatasetTimeSeries'>


In [6]:

# pioneer_dataset = get_ml_image(pioneer_img_list)
    
# MEAN = np.nanmean(pioneer_dataset,(0,2,3))
# STD = np.nanstd(pioneer_dataset, (0,2,3))
# print(MEAN, STD)

In [7]:
from sklearn.model_selection import KFold
import torch.optim as optim
from dl.model import EncoderCNN, LSTMRegression

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=39)

in_channel = 5
num_epochs = 100
batch_size = 32

# Initialize an empty list to store fold-wise performance
fold_accuracies = []

# Initialize a new model for each fold
embed_size = 256
# encoder = EncoderCNN(in_channel, embed_size)
# decoder = LSTMRegression(embed_size, 64)
model = ConvLSTMRegression(in_channel, 256, 128, 5, 1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# decoder.to(device)

criterion = torch.nn.MSELoss()  # Mean Squared Error loss function
# optimizer = optim.Adam(list(conv.parameters()) + list(deconv.parameters()), lr=0.001)  # Adam optimizer
# params = list(decoder.parameters()) + list(encoder.parameters())
params = list(model.parameters())
optimizer = optim.Adam(params = params, lr=0.001)  # Adam optimizer

# optimizer.zero_grad()
  

In [8]:
model = train_with_cross_validation(model, train_val_dataset, batch_size, num_epochs, optimizer, criterion)

validation mse is [2511.2021484375, 2406.32470703125, 2414.206298828125, 2620.45263671875, 2431.562744140625, 2371.388427734375, 2967.73779296875, 2301.8359375, 2267.20849609375, 3175.4599609375, 1634.531982421875, 1896.4498291015625, 1940.7796630859375, 3040.39501953125, 1784.591064453125, 2462.76513671875, 2196.28857421875, 2170.624267578125, 2261.906982421875, 2930.279541015625]
validation mse is [2250.3447265625, 1920.94140625, 2120.612548828125, 1724.341064453125, 1994.77978515625, 2810.070556640625, 1398.7020263671875, 2619.2607421875, 2361.29833984375, 2544.627685546875, 2364.96630859375, 2077.87158203125, 1918.868408203125, 1744.46240234375, 2771.3896484375, 2352.07275390625, 1991.145751953125, 2345.749267578125, 2366.456787109375, 2332.54541015625]
validation mse is [2409.133544921875, 2490.9365234375, 2390.50634765625, 2721.398193359375, 2199.03271484375, 2052.686279296875, 2013.3387451171875, 2230.34130859375, 2024.6546630859375, 1778.324462890625, 2577.15380859375, 2058.536

In [9]:
model_name = "path/lstm_model_DOY171_state.pth"

In [ ]:
torch.save(model.state_dict(), model_name)

In [ ]:
          
for train_indices, val_indices in kf.split(train_val_dataset):
    train_dataset = torch.utils.data.Subset(train_val_dataset, train_indices)
    val_dataset = torch.utils.data.Subset(train_val_dataset, val_indices)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    encoder.train()
    decoder.train()
    # Train your model on the training data
    for epoch in range(num_epochs):
        # print('start')
        for i, data in enumerate(train_loader, 0):
            # Get inputs and labels
            inputs, labels = data
            # print(inputs.shape)
            # print(labels.shape)
            # labels = labels.unsqueeze(1) #change to (32,1)
            labels = labels.view(-1, 1)
            # print(labels.shape)
            
            # Move batch of images and captions to GPU if CUDA is available.
            inputs = inputs.to(device)
            labels = labels.to(device)
            # Zero the gradients.
            decoder.zero_grad()
            encoder.zero_grad()
            features = None
            # Pass the inputs through the CNN-RNN model.
            for step in range(inputs.shape[1]):
                # print(step)
                feature = encoder(inputs[:,step,:,:])
                feature = feature.unsqueeze(1)

                if step == 0:
                    features = feature
                else:
                    features = torch.cat([features, feature], dim=1)
                # print(features.shape)
            
            outputs = decoder(features)
            del features
            # # Calculate the batch loss.
            # loss = criterion(outputs, labels)
            # # Backward pass.
            # loss.backward()
            
            # # Update the parameters in the optimizer.
            # optimizer.step()
            
            # Compute loss
            if  outputs.shape[0] == labels.shape[0]:
                loss = criterion(outputs[:,-1].squeeze(), labels[:,-1].squeeze())
                
                # Backward pass and optimization
                loss.backward()
                # update weights
                optimizer.step()
            else:
                print('outputs shape is ', outputs.shape, ' the first value is ', outputs[0])
                print('labels shape is ', labels.shape, ' the first label is ', labels[0])
            
            # # Print statistics
            # running_loss += loss.item()
            # train_loss +=  loss.item()
            # if i % 100 == 99:    # Print every 100 mini-batches
            #     print('[%d, %5d] loss: %.3f' %
            #         (epoch + 1, i + 1, running_loss))
            #     running_loss = 0.0


    # Evaluate the model on the validation data
    encoder.eval()
    decoder.eval()
    val_accuracy = []
    for i, data in enumerate(val_loader, 0):
        inputs, labels = data
        labels = labels.view(-1, 1)

        # Perform forward and backward passes through the neural network
        # optimizer.zero_grad()
        features = encoder(inputs)
        outputs = decoder(features)
        loss = criterion(outputs, labels)
        val_accuracy.append(loss.detach().numpy())
        
        # Evaluation loop
    val_accuracy = np.mean(val_accuracy)
    print(f'validation mse is {val_accuracy}')
    fold_accuracies.append(val_accuracy)

# Calculate the average accuracy across all folds
average_accuracy = np.mean(fold_accuracies)
print(f'The final validation mse is {average_accuracy}')

In [ ]:
print(np.sqrt(average_accuracy))

In [ ]:
print(outputs)
print(labels)

In [ ]:
print(np.sqrt(average_accuracy))

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



# Evaluate the model on the validation data
# model.eval()

In [ ]:

test_accuracy = []
test_pred = []
test_truth = []

for i, data in enumerate(test_loader, 0):
    inputs, labels = data
    labels = labels.view(-1, 1)

    # Perform forward and backward passes through the neural network
    optimizer.zero_grad()
    outputs = model(inputs)
    # print(outputs.shape)

    test_pred.extend(outputs.squeeze().tolist())
    test_truth.extend(labels.squeeze().tolist())
    loss = criterion(outputs, labels)
    # print(outputs.squeeze(), labels.squeeze())
    test_accuracy.append(loss.detach().numpy())
    
    # Evaluation loop
test_accuracy2 = np.mean(test_pred)
print(f'validation mse is {test_accuracy2}')

   

In [ ]:
yield_data = np.array(pioneer_yield_list)
irrigate_data = np.array(irrigate_type_list)

gt2 = yield_data[test_indices]

In [ ]:
print(test_truth - gt2)

In [ ]:
from ml.ml_predict import plot_result, plot_result_separate


In [ ]:
plot_result(test_truth, test_pred, 'test.png')

In [ ]:
print(test_indices)

In [ ]:
deficit_indices = np.where(irrigate_data == 1)[0]
full_indices = np.where(irrigate_data == 0)[0]

In [ ]:
np.in1d(test_indices, deficit_indices)


In [ ]:
print(irrigate_data)

In [ ]:
plot_result_separate(np.array(test_truth), np.array(test_pred), test_indices, irrigate_data, 'test2.png')

In [ ]:
print(all_outputs)

In [ ]:
a,b = test_dataset
print(test_dataset)